In [2]:
import numpy as np 
import pandas as pd
import time
import datetime
import gc
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2, SelectPercentile
import math
from sklearn.metrics import f1_score
import jieba
import jieba.posseg as psg
from collections import Counter
import functools
from gensim.models import word2vec
import Levenshtein


In [29]:
train_dataset = pd.read_csv('../temp/train_offline_df.csv')
valid_dataset = pd.read_csv('../temp/valid_offline_df.csv')



In [30]:
fea = [
    'query_prediction_number', 'query_prediction_max', 'query_prediction_min', 'query_prediction_mean', 'query_prediction_std',
       'prefix_count', 'prefix_rate',
 'title_count', 'title_rate', 'tag_count', 'tag_rate',
 'query_prediction_count', 'query_prediction_rate', 'prefix_title_count',
 'prefix_title_rate',  'prefix_tag_count', 'prefix_tag_rate',
 'title_tag_count', 'title_tag_rate',
    'prefix_click_number', 'title_click_number', 'query_prediction_click_number', 'prefix_tag_click_number', 
    'prefix_title_click_number', 'title_tag_click_number',
    'is_title_in_query', 'is_prefix_in_title', 
    'title_tag_types', 'prefix_tag_types', 'tag_title_types', 'tag_prefix_types',
 'title_prefix_types', 'prefix_title_types', 'tag_query_prediction_types', 'title_query_prediction_types',
      'prefix_len', 'title_len',
 'query_prediction_key_len_max', 'query_prediction_key_len_min',
 'query_prediction_key_len_mean', 'query_prediction_key_len_std',
 'len_title-prefix', 'len_prefix/title', 'len_mean-title', 'len_mean/title',
    'q_t_word_match', 'q_t_jaccard', 'q_t_common_words',
 'q_t_total_unique_words', 'q_t_wc_diff', 'q_t_wc_ratio',
 'q_t_wc_diff_unique', 'q_t_wc_ratio_unique', 'q_t_tfidf_word_match_share',
 'p_t_word_match', 'p_t_jaccard', 'p_t_common_words',
 'p_t_total_unique_words', 'p_t_wc_diff', 'p_t_wc_ratio',
 'p_t_wc_diff_unique', 'p_t_wc_ratio_unique', 'p_t_tfidf_word_match_share',
 'p_q_word_match', 'p_q_jaccard', 'p_q_common_words',
 'p_q_total_unique_words', 'p_q_wc_diff', 'p_q_wc_ratio',
 'p_q_wc_diff_unique', 'p_q_wc_ratio_unique', 'p_q_tfidf_word_match_share',
    'title_prefix_dot_similarity',
 'title_query_dot_similarity', 'title_prefix_norm_similarity',
 'title_query_norm_similarity', 'title_prefix_cosine_similarity',
 'title_query_cosine_similarity',
    'title_query_dot_similarity_max', 'title_query_dot_similarity_min',
 'title_query_dot_similarity_mean', 'title_query_dot_similarity_std',
    'title_query_norm_similarity_min', 'title_query_norm_similarity_mean',
 'title_query_norm_similarity_std', 'title_prefix_cosine_similarity',
    'title_query_cosine_similarity_max', 'title_query_cosine_similarity_min',
 'title_query_cosine_similarity_mean', 'title_query_cosine_similarity_std',
    'title_prefix_leven', 'title_prefix_leven_rate',
 'title_query_leven_sum', 'title_query_leven_max', 'title_query_leven_min',
 'title_query_leven_mean', 'title_query_leven_std',
      ]



In [31]:
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=127, max_depth=-1, n_estimators=5000, objective='binary',
    subsample=0.8, colsample_bytree=1, subsample_freq=1,
    learning_rate=0.01, random_state=2018, n_jobs=-1
)

valid_dataset['predicted_score'] = 0

lgb_model.fit(train_dataset[fea], train_dataset['label'], eval_set=[(train_dataset[fea], train_dataset['label']),
                            (valid_dataset[fea], valid_dataset['label'])], early_stopping_rounds=50, eval_metric='auc')
valid_pred = lgb_model.predict_proba(valid_dataset[fea], num_iteration=lgb_model.best_iteration_)[:, 1]
print(np.mean(valid_pred))

fscore = lgb_model.booster_.feature_importance()
feaNames = lgb_model.booster_.feature_name()
scoreDf = pd.DataFrame(index=feaNames, columns=['importance'], data=fscore)
print(scoreDf.sort_index(by=['importance'], ascending=False))



[1]	valid_0's auc: 0.870406	valid_1's auc: 0.860186
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's auc: 0.871064	valid_1's auc: 0.864261
[3]	valid_0's auc: 0.871523	valid_1's auc: 0.865351
[4]	valid_0's auc: 0.871578	valid_1's auc: 0.864863
[5]	valid_0's auc: 0.871699	valid_1's auc: 0.865227
[6]	valid_0's auc: 0.871825	valid_1's auc: 0.865162
[7]	valid_0's auc: 0.871876	valid_1's auc: 0.864975
[8]	valid_0's auc: 0.872001	valid_1's auc: 0.865081
[9]	valid_0's auc: 0.872162	valid_1's auc: 0.865358
[10]	valid_0's auc: 0.872415	valid_1's auc: 0.865703
[11]	valid_0's auc: 0.872463	valid_1's auc: 0.866001
[12]	valid_0's auc: 0.872482	valid_1's auc: 0.86582
[13]	valid_0's auc: 0.872503	valid_1's auc: 0.865926
[14]	valid_0's auc: 0.872603	valid_1's auc: 0.866195
[15]	valid_0's auc: 0.872652	valid_1's auc: 0.866163
[16]	valid_0's auc: 0.872677	valid_1's auc: 0.866053
[17]	valid_0's auc: 0.87278	valid_1's auc: 0.866324
[18]	valid_0's auc: 0.872836	valid_1's auc: 0.86

[155]	valid_0's auc: 0.876882	valid_1's auc: 0.871465
[156]	valid_0's auc: 0.876901	valid_1's auc: 0.871486
[157]	valid_0's auc: 0.876926	valid_1's auc: 0.871507
[158]	valid_0's auc: 0.876948	valid_1's auc: 0.871534
[159]	valid_0's auc: 0.876969	valid_1's auc: 0.871557
[160]	valid_0's auc: 0.876989	valid_1's auc: 0.871575
[161]	valid_0's auc: 0.877013	valid_1's auc: 0.871599
[162]	valid_0's auc: 0.877031	valid_1's auc: 0.871613
[163]	valid_0's auc: 0.877057	valid_1's auc: 0.87165
[164]	valid_0's auc: 0.877071	valid_1's auc: 0.871673
[165]	valid_0's auc: 0.877093	valid_1's auc: 0.871709
[166]	valid_0's auc: 0.877109	valid_1's auc: 0.871727
[167]	valid_0's auc: 0.877127	valid_1's auc: 0.871736
[168]	valid_0's auc: 0.87715	valid_1's auc: 0.871766
[169]	valid_0's auc: 0.877167	valid_1's auc: 0.871782
[170]	valid_0's auc: 0.877185	valid_1's auc: 0.871811
[171]	valid_0's auc: 0.877202	valid_1's auc: 0.871833
[172]	valid_0's auc: 0.877217	valid_1's auc: 0.871853
[173]	valid_0's auc: 0.877241	

[308]	valid_0's auc: 0.87937	valid_1's auc: 0.873253
[309]	valid_0's auc: 0.879385	valid_1's auc: 0.873262
[310]	valid_0's auc: 0.879397	valid_1's auc: 0.873262
[311]	valid_0's auc: 0.879409	valid_1's auc: 0.873265
[312]	valid_0's auc: 0.879423	valid_1's auc: 0.873276
[313]	valid_0's auc: 0.879436	valid_1's auc: 0.873286
[314]	valid_0's auc: 0.87945	valid_1's auc: 0.873281
[315]	valid_0's auc: 0.879461	valid_1's auc: 0.873287
[316]	valid_0's auc: 0.879474	valid_1's auc: 0.873297
[317]	valid_0's auc: 0.879488	valid_1's auc: 0.873304
[318]	valid_0's auc: 0.8795	valid_1's auc: 0.873309
[319]	valid_0's auc: 0.879513	valid_1's auc: 0.873309
[320]	valid_0's auc: 0.879529	valid_1's auc: 0.873316
[321]	valid_0's auc: 0.879541	valid_1's auc: 0.873306
[322]	valid_0's auc: 0.879552	valid_1's auc: 0.873305
[323]	valid_0's auc: 0.879565	valid_1's auc: 0.873312
[324]	valid_0's auc: 0.879578	valid_1's auc: 0.873315
[325]	valid_0's auc: 0.879592	valid_1's auc: 0.87332
[326]	valid_0's auc: 0.879605	val

[461]	valid_0's auc: 0.881363	valid_1's auc: 0.874123
[462]	valid_0's auc: 0.881378	valid_1's auc: 0.874131
[463]	valid_0's auc: 0.881391	valid_1's auc: 0.874152
[464]	valid_0's auc: 0.881401	valid_1's auc: 0.874154
[465]	valid_0's auc: 0.881413	valid_1's auc: 0.874158
[466]	valid_0's auc: 0.881425	valid_1's auc: 0.874165
[467]	valid_0's auc: 0.881437	valid_1's auc: 0.874174
[468]	valid_0's auc: 0.881448	valid_1's auc: 0.874181
[469]	valid_0's auc: 0.881458	valid_1's auc: 0.874178
[470]	valid_0's auc: 0.881469	valid_1's auc: 0.874181
[471]	valid_0's auc: 0.881481	valid_1's auc: 0.87418
[472]	valid_0's auc: 0.881492	valid_1's auc: 0.87418
[473]	valid_0's auc: 0.881505	valid_1's auc: 0.874179
[474]	valid_0's auc: 0.881516	valid_1's auc: 0.87418
[475]	valid_0's auc: 0.88153	valid_1's auc: 0.874186
[476]	valid_0's auc: 0.881543	valid_1's auc: 0.87419
[477]	valid_0's auc: 0.881554	valid_1's auc: 0.874188
[478]	valid_0's auc: 0.881564	valid_1's auc: 0.874193
[479]	valid_0's auc: 0.881578	val

[614]	valid_0's auc: 0.882865	valid_1's auc: 0.874464
[615]	valid_0's auc: 0.882873	valid_1's auc: 0.874468
[616]	valid_0's auc: 0.882881	valid_1's auc: 0.874471
[617]	valid_0's auc: 0.882889	valid_1's auc: 0.874463
[618]	valid_0's auc: 0.882898	valid_1's auc: 0.874462
[619]	valid_0's auc: 0.882905	valid_1's auc: 0.874465
[620]	valid_0's auc: 0.882914	valid_1's auc: 0.874471
[621]	valid_0's auc: 0.882922	valid_1's auc: 0.874471
[622]	valid_0's auc: 0.882929	valid_1's auc: 0.874471
[623]	valid_0's auc: 0.882937	valid_1's auc: 0.87447
[624]	valid_0's auc: 0.882944	valid_1's auc: 0.874474
[625]	valid_0's auc: 0.882952	valid_1's auc: 0.874475
[626]	valid_0's auc: 0.882961	valid_1's auc: 0.874479
[627]	valid_0's auc: 0.882969	valid_1's auc: 0.874477
[628]	valid_0's auc: 0.882976	valid_1's auc: 0.874475
[629]	valid_0's auc: 0.882984	valid_1's auc: 0.874474
[630]	valid_0's auc: 0.882993	valid_1's auc: 0.874476
[631]	valid_0's auc: 0.883	valid_1's auc: 0.874476
[632]	valid_0's auc: 0.883008	va

/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [32]:
valid_dataset['predicted_score'] = valid_pred

train_prefix_set = set(train_dataset['prefix'])

valid_dataset['is_prefix_in_train'] = valid_dataset['prefix'].map(lambda x : 1 if x in train_prefix_set else 0)
print(np.mean(valid_dataset[valid_dataset.is_prefix_in_train == 1]['predicted_score']))
print(np.mean(valid_dataset[valid_dataset.is_prefix_in_train == 0]['predicted_score']))



0.3716900336467934
0.4594389031819946


In [34]:
#直接搜索最佳阈值

yuzhi_dict = {}
# 定义搜索方法获取最佳F1对应的阈值
for yuzhi in range(350, 700, 2):
    real_yuzhi = yuzhi / 1000
    valid_dataset['predicted_label'] = valid_dataset['predicted_score'].map(lambda x : 1 if x > real_yuzhi else 0)
    f1 = f1_score(valid_dataset['label'], valid_dataset['predicted_label'])
    yuzhi_dict[str(real_yuzhi)] = f1
print(yuzhi_dict)



{'0.35': 0.7381686873759477, '0.352': 0.7385588045118183, '0.354': 0.7389353995250318, '0.356': 0.7386396387741377, '0.358': 0.7382721392675947, '0.36': 0.7381732344156471, '0.362': 0.7381004129537058, '0.364': 0.7382173618503822, '0.366': 0.7381437299970904, '0.368': 0.7384996719399285, '0.37': 0.7384660307572514, '0.372': 0.7384202827888834, '0.374': 0.7383300781250001, '0.376': 0.738674763700475, '0.378': 0.7387369711833232, '0.38': 0.7385757478921364, '0.382': 0.7389310718864016, '0.384': 0.7387494137105334, '0.386': 0.738346645564925, '0.388': 0.7387824880453904, '0.39': 0.7388478142212078, '0.392': 0.7388345173630086, '0.394': 0.73915857605178, '0.396': 0.7395293179098524, '0.398': 0.7397998652459263, '0.4': 0.7403550058833838, '0.402': 0.7400140505820956, '0.404': 0.7403722963297912, '0.406': 0.7402022437993662, '0.408': 0.7402106536310034, '0.41': 0.7399626281500934, '0.412': 0.7399939357186173, '0.414': 0.7399069108570272, '0.416': 0.7399787158566867, '0.418': 0.73996447602131

In [40]:
valid_prefix0_df = valid_dataset[valid_dataset.is_prefix_in_train == 1].copy()

#定义调整函数
def resultAdjustment(result_df, t):
    result_df_temp = result_df.copy()
    result_df_temp['x'] = result_df_temp.predicted_score.map(lambda x: -(math.log(((1 - x) / x), math.e)))
    result_df_temp['adjust_result'] = result_df_temp.x.map(lambda x: 1 / (1 + math.exp(-(x + t)))) 
    print(result_df_temp['adjust_result'].mean())
    return result_df_temp['adjust_result']

print('original mean : ', valid_prefix0_df['predicted_score'].mean())
valid_df_after = resultAdjustment(valid_prefix0_df, 0.64405)


original mean :  0.3716900336467934
0.45940527673569165


In [41]:
valid_dataset['predicted_score'][valid_dataset.is_prefix_in_train == 1] = valid_df_after
print(np.mean(valid_dataset['predicted_score'][valid_dataset.is_prefix_in_train == 0]))
print(np.mean(valid_dataset['predicted_score'][valid_dataset.is_prefix_in_train == 1]))



0.4594389031819946
0.45940527673569165


/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
yuzhi_dict = {}
# 定义搜索方法获取最佳F1对应的阈值
for yuzhi in range(350, 700, 2):
    real_yuzhi = yuzhi / 1000
    valid_dataset['predicted_label'] = valid_dataset['predicted_score'].map(lambda x : 1 if x > real_yuzhi else 0)
    f1 = f1_score(valid_dataset['label'], valid_dataset['predicted_label'])
    yuzhi_dict[str(real_yuzhi)] = f1
print(yuzhi_dict)



{'0.35': 0.7210911074740862, '0.352': 0.7216783216783218, '0.354': 0.7221407785046933, '0.356': 0.7225826247041289, '0.358': 0.7228910369068543, '0.36': 0.7232516174464151, '0.362': 0.7239196135095189, '0.364': 0.7246850828729282, '0.366': 0.7250309899061449, '0.368': 0.7257080754984807, '0.37': 0.7258637534416912, '0.372': 0.7259582230329457, '0.374': 0.7261018761976915, '0.376': 0.7263432819161998, '0.378': 0.7268539225333215, '0.38': 0.7273175542406312, '0.382': 0.7280280235325819, '0.384': 0.7284219997750535, '0.386': 0.7282562149248349, '0.388': 0.7288955169920464, '0.39': 0.7296844583276745, '0.392': 0.730348800290289, '0.394': 0.7308496138119037, '0.396': 0.7312224193621765, '0.398': 0.7317295425551305, '0.4': 0.7328931435490873, '0.402': 0.7329135661133372, '0.404': 0.7332186675839926, '0.406': 0.7337362410092608, '0.408': 0.7341789636041345, '0.41': 0.7342513782206537, '0.412': 0.7346627538877464, '0.414': 0.7349952549591465, '0.416': 0.7354066652751687, '0.418': 0.73546349619

In [43]:
valid_dataset['predicted_label'] = valid_dataset['predicted_score'].map(lambda x : 1 if x > 0.476 else 0)


In [44]:
print(np.mean(valid_dataset['label'][valid_dataset.is_prefix_in_train == 0]))
print(np.mean(valid_dataset['label'][valid_dataset.is_prefix_in_train == 1]))
print(np.mean(valid_dataset['predicted_label'][valid_dataset.is_prefix_in_train == 0]))
print(np.mean(valid_dataset['predicted_label'][valid_dataset.is_prefix_in_train == 1]))


0.3741522590505301
0.37105056790872987
0.45849651351609516
0.43350787989173056


In [49]:
#测试五折效果
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=127, max_depth=-1, n_estimators=5000, objective='binary',
    subsample=0.8, colsample_bytree=1, subsample_freq=1,
    learning_rate=0.01, random_state=2018, n_jobs=-1
)

valid_dataset['predicted_score'] = 0

skf = StratifiedKFold(n_splits=5, random_state=2018, shuffle=True)
best_score = []
for index, (train_index, test_index) in enumerate(skf.split(train_dataset, train_dataset['label'])):
    lgb_model.fit(train_dataset[fea].iloc[train_index], train_dataset['label'][train_index],
                  eval_set=[(train_dataset[fea].iloc[train_index], train_dataset['label'][train_index]),
                            (valid_dataset[fea], valid_dataset['label'])], early_stopping_rounds=50, eval_metric='auc')
    best_score.append(lgb_model.best_score_['valid_1']['auc'])
    print(best_score)
    valid_pred = lgb_model.predict_proba(valid_dataset[fea], num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', valid_pred.mean())
    valid_dataset['predicted_score'] = valid_dataset['predicted_score'] + valid_pred
print(np.mean(best_score))

valid_dataset['predicted_score'] = valid_dataset['predicted_score'] / 5
mean = valid_dataset['predicted_score'].mean()
print('mean:', mean)



[1]	valid_0's auc: 0.87044	valid_1's auc: 0.862853
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's auc: 0.871207	valid_1's auc: 0.865146
[3]	valid_0's auc: 0.87148	valid_1's auc: 0.865115
[4]	valid_0's auc: 0.871798	valid_1's auc: 0.865457
[5]	valid_0's auc: 0.871968	valid_1's auc: 0.865644
[6]	valid_0's auc: 0.87203	valid_1's auc: 0.865574
[7]	valid_0's auc: 0.872116	valid_1's auc: 0.865552
[8]	valid_0's auc: 0.872205	valid_1's auc: 0.865926
[9]	valid_0's auc: 0.872302	valid_1's auc: 0.866207
[10]	valid_0's auc: 0.872404	valid_1's auc: 0.86649
[11]	valid_0's auc: 0.872481	valid_1's auc: 0.866477
[12]	valid_0's auc: 0.872546	valid_1's auc: 0.866643
[13]	valid_0's auc: 0.872592	valid_1's auc: 0.86677
[14]	valid_0's auc: 0.872641	valid_1's auc: 0.866697
[15]	valid_0's auc: 0.872731	valid_1's auc: 0.866785
[16]	valid_0's auc: 0.872824	valid_1's auc: 0.867015
[17]	valid_0's auc: 0.872863	valid_1's auc: 0.867103
[18]	valid_0's auc: 0.872906	valid_1's auc: 0.86727

[155]	valid_0's auc: 0.877013	valid_1's auc: 0.871881
[156]	valid_0's auc: 0.877028	valid_1's auc: 0.871888
[157]	valid_0's auc: 0.877047	valid_1's auc: 0.871893
[158]	valid_0's auc: 0.87707	valid_1's auc: 0.871919
[159]	valid_0's auc: 0.877088	valid_1's auc: 0.871937
[160]	valid_0's auc: 0.877107	valid_1's auc: 0.871961
[161]	valid_0's auc: 0.877132	valid_1's auc: 0.871978
[162]	valid_0's auc: 0.877147	valid_1's auc: 0.872005
[163]	valid_0's auc: 0.877174	valid_1's auc: 0.87204
[164]	valid_0's auc: 0.877191	valid_1's auc: 0.872054
[165]	valid_0's auc: 0.877214	valid_1's auc: 0.872067
[166]	valid_0's auc: 0.877237	valid_1's auc: 0.872083
[167]	valid_0's auc: 0.877257	valid_1's auc: 0.872126
[168]	valid_0's auc: 0.877282	valid_1's auc: 0.872161
[169]	valid_0's auc: 0.877298	valid_1's auc: 0.872175
[170]	valid_0's auc: 0.877322	valid_1's auc: 0.872195
[171]	valid_0's auc: 0.877343	valid_1's auc: 0.872221
[172]	valid_0's auc: 0.877364	valid_1's auc: 0.872239
[173]	valid_0's auc: 0.877387	

[308]	valid_0's auc: 0.879582	valid_1's auc: 0.873769
[309]	valid_0's auc: 0.879593	valid_1's auc: 0.873768
[310]	valid_0's auc: 0.879608	valid_1's auc: 0.87378
[311]	valid_0's auc: 0.879621	valid_1's auc: 0.873783
[312]	valid_0's auc: 0.879636	valid_1's auc: 0.873777
[313]	valid_0's auc: 0.879649	valid_1's auc: 0.873763
[314]	valid_0's auc: 0.879664	valid_1's auc: 0.873754
[315]	valid_0's auc: 0.879678	valid_1's auc: 0.873772
[316]	valid_0's auc: 0.879692	valid_1's auc: 0.873771
[317]	valid_0's auc: 0.879705	valid_1's auc: 0.873767
[318]	valid_0's auc: 0.879718	valid_1's auc: 0.873779
[319]	valid_0's auc: 0.879733	valid_1's auc: 0.873791
[320]	valid_0's auc: 0.87975	valid_1's auc: 0.873804
[321]	valid_0's auc: 0.879765	valid_1's auc: 0.873798
[322]	valid_0's auc: 0.879778	valid_1's auc: 0.873803
[323]	valid_0's auc: 0.87979	valid_1's auc: 0.873809
[324]	valid_0's auc: 0.879804	valid_1's auc: 0.873811
[325]	valid_0's auc: 0.879819	valid_1's auc: 0.873812
[326]	valid_0's auc: 0.879834	v

[461]	valid_0's auc: 0.881681	valid_1's auc: 0.874496
[462]	valid_0's auc: 0.881694	valid_1's auc: 0.874512
[463]	valid_0's auc: 0.881706	valid_1's auc: 0.874516
[464]	valid_0's auc: 0.881718	valid_1's auc: 0.874524
[465]	valid_0's auc: 0.881729	valid_1's auc: 0.874528
[466]	valid_0's auc: 0.881741	valid_1's auc: 0.87453
[467]	valid_0's auc: 0.881753	valid_1's auc: 0.87453
[468]	valid_0's auc: 0.881766	valid_1's auc: 0.874532
[469]	valid_0's auc: 0.881778	valid_1's auc: 0.874528
[470]	valid_0's auc: 0.88179	valid_1's auc: 0.874532
[471]	valid_0's auc: 0.881801	valid_1's auc: 0.874537
[472]	valid_0's auc: 0.881813	valid_1's auc: 0.87454
[473]	valid_0's auc: 0.881823	valid_1's auc: 0.874537
[474]	valid_0's auc: 0.881836	valid_1's auc: 0.874548
[475]	valid_0's auc: 0.881848	valid_1's auc: 0.874556
[476]	valid_0's auc: 0.881861	valid_1's auc: 0.874562
[477]	valid_0's auc: 0.881875	valid_1's auc: 0.874568
[478]	valid_0's auc: 0.881886	valid_1's auc: 0.874576
[479]	valid_0's auc: 0.881897	va

[614]	valid_0's auc: 0.883291	valid_1's auc: 0.874866
[615]	valid_0's auc: 0.883299	valid_1's auc: 0.874866
[616]	valid_0's auc: 0.883309	valid_1's auc: 0.874868
[617]	valid_0's auc: 0.883316	valid_1's auc: 0.874873
[618]	valid_0's auc: 0.883326	valid_1's auc: 0.87487
[619]	valid_0's auc: 0.883334	valid_1's auc: 0.874871
[620]	valid_0's auc: 0.88334	valid_1's auc: 0.874868
[621]	valid_0's auc: 0.88335	valid_1's auc: 0.87487
[622]	valid_0's auc: 0.883359	valid_1's auc: 0.874877
[623]	valid_0's auc: 0.883368	valid_1's auc: 0.874875
[624]	valid_0's auc: 0.883378	valid_1's auc: 0.874876
[625]	valid_0's auc: 0.883387	valid_1's auc: 0.874876
[626]	valid_0's auc: 0.883396	valid_1's auc: 0.874877
[627]	valid_0's auc: 0.883404	valid_1's auc: 0.874877
[628]	valid_0's auc: 0.883411	valid_1's auc: 0.874876
[629]	valid_0's auc: 0.88342	valid_1's auc: 0.874876
[630]	valid_0's auc: 0.883428	valid_1's auc: 0.874873
[631]	valid_0's auc: 0.883438	valid_1's auc: 0.874877
[632]	valid_0's auc: 0.883447	val

...

[91]	valid_0's auc: 0.875399	valid_1's auc: 0.869801
[92]	valid_0's auc: 0.875426	valid_1's auc: 0.869832
[93]	valid_0's auc: 0.875452	valid_1's auc: 0.86985
[94]	valid_0's auc: 0.875473	valid_1's auc: 0.869877
[95]	valid_0's auc: 0.875495	valid_1's auc: 0.869916
[96]	valid_0's auc: 0.875518	valid_1's auc: 0.869941
[97]	valid_0's auc: 0.875546	valid_1's auc: 0.869967
[98]	valid_0's auc: 0.875567	valid_1's auc: 0.870003
[99]	valid_0's auc: 0.875592	valid_1's auc: 0.870012
[100]	valid_0's auc: 0.875615	valid_1's auc: 0.870018
[101]	valid_0's auc: 0.875641	valid_1's auc: 0.870061
[102]	valid_0's auc: 0.875661	valid_1's auc: 0.870082
[103]	valid_0's auc: 0.875684	valid_1's auc: 0.870125
[104]	valid_0's auc: 0.875706	valid_1's auc: 0.870134
[105]	valid_0's auc: 0.87573	valid_1's auc: 0.870154
[106]	valid_0's auc: 0.875758	valid_1's auc: 0.870178
[107]	valid_0's auc: 0.875785	valid_1's auc: 0.870199
[108]	valid_0's auc: 0.875808	valid_1's auc: 0.870227
[109]	valid_0's auc: 0.875822	valid_1's

[244]	valid_0's auc: 0.87845	valid_1's auc: 0.872535
[245]	valid_0's auc: 0.878466	valid_1's auc: 0.872552
[246]	valid_0's auc: 0.878486	valid_1's auc: 0.872562
[247]	valid_0's auc: 0.878502	valid_1's auc: 0.872568
[248]	valid_0's auc: 0.878518	valid_1's auc: 0.87259
[249]	valid_0's auc: 0.878532	valid_1's auc: 0.872603
[250]	valid_0's auc: 0.878551	valid_1's auc: 0.872615
[251]	valid_0's auc: 0.878567	valid_1's auc: 0.872628
[252]	valid_0's auc: 0.878585	valid_1's auc: 0.872639
[253]	valid_0's auc: 0.878602	valid_1's auc: 0.872652
[254]	valid_0's auc: 0.878619	valid_1's auc: 0.872669
[255]	valid_0's auc: 0.878635	valid_1's auc: 0.872672
[256]	valid_0's auc: 0.87865	valid_1's auc: 0.872684
[257]	valid_0's auc: 0.878664	valid_1's auc: 0.872709
[258]	valid_0's auc: 0.878679	valid_1's auc: 0.872716
[259]	valid_0's auc: 0.878695	valid_1's auc: 0.872719
[260]	valid_0's auc: 0.87871	valid_1's auc: 0.872726
[261]	valid_0's auc: 0.878728	valid_1's auc: 0.872736
[262]	valid_0's auc: 0.878745	va

[397]	valid_0's auc: 0.880681	valid_1's auc: 0.873503
[398]	valid_0's auc: 0.880693	valid_1's auc: 0.873506
[399]	valid_0's auc: 0.880707	valid_1's auc: 0.873505
[400]	valid_0's auc: 0.88072	valid_1's auc: 0.873514
[401]	valid_0's auc: 0.880733	valid_1's auc: 0.873513
[402]	valid_0's auc: 0.880746	valid_1's auc: 0.873512
[403]	valid_0's auc: 0.880759	valid_1's auc: 0.873516
[404]	valid_0's auc: 0.880773	valid_1's auc: 0.873525
[405]	valid_0's auc: 0.880784	valid_1's auc: 0.873531
[406]	valid_0's auc: 0.880798	valid_1's auc: 0.873535
[407]	valid_0's auc: 0.880811	valid_1's auc: 0.873542
[408]	valid_0's auc: 0.880824	valid_1's auc: 0.873554
[409]	valid_0's auc: 0.880838	valid_1's auc: 0.873565
[410]	valid_0's auc: 0.880851	valid_1's auc: 0.873569
[411]	valid_0's auc: 0.880864	valid_1's auc: 0.873569
[412]	valid_0's auc: 0.880878	valid_1's auc: 0.873576
[413]	valid_0's auc: 0.880891	valid_1's auc: 0.873584
[414]	valid_0's auc: 0.880905	valid_1's auc: 0.873586
[415]	valid_0's auc: 0.880922

[550]	valid_0's auc: 0.882526	valid_1's auc: 0.874053
[551]	valid_0's auc: 0.882536	valid_1's auc: 0.87405
[552]	valid_0's auc: 0.882546	valid_1's auc: 0.874054
[553]	valid_0's auc: 0.882555	valid_1's auc: 0.874056
[554]	valid_0's auc: 0.882565	valid_1's auc: 0.874056
[555]	valid_0's auc: 0.882575	valid_1's auc: 0.874063
[556]	valid_0's auc: 0.882584	valid_1's auc: 0.874069
[557]	valid_0's auc: 0.882594	valid_1's auc: 0.874067
[558]	valid_0's auc: 0.882602	valid_1's auc: 0.874068
[559]	valid_0's auc: 0.882612	valid_1's auc: 0.874072
[560]	valid_0's auc: 0.882622	valid_1's auc: 0.874075
[561]	valid_0's auc: 0.882634	valid_1's auc: 0.874075
[562]	valid_0's auc: 0.882645	valid_1's auc: 0.874089
[563]	valid_0's auc: 0.882654	valid_1's auc: 0.87409
[564]	valid_0's auc: 0.882665	valid_1's auc: 0.874088
[565]	valid_0's auc: 0.882674	valid_1's auc: 0.87409
[566]	valid_0's auc: 0.882684	valid_1's auc: 0.874093
[567]	valid_0's auc: 0.882694	valid_1's auc: 0.874091
[568]	valid_0's auc: 0.882705	v

[703]	valid_0's auc: 0.883886	valid_1's auc: 0.87416
[704]	valid_0's auc: 0.883894	valid_1's auc: 0.87416
[705]	valid_0's auc: 0.8839	valid_1's auc: 0.87416
[706]	valid_0's auc: 0.883907	valid_1's auc: 0.874161
[707]	valid_0's auc: 0.883914	valid_1's auc: 0.874159
[708]	valid_0's auc: 0.883921	valid_1's auc: 0.874159
[709]	valid_0's auc: 0.883929	valid_1's auc: 0.874162
[710]	valid_0's auc: 0.883938	valid_1's auc: 0.874159
[711]	valid_0's auc: 0.883946	valid_1's auc: 0.874161
[712]	valid_0's auc: 0.883955	valid_1's auc: 0.87416
[713]	valid_0's auc: 0.883962	valid_1's auc: 0.874159
[714]	valid_0's auc: 0.883972	valid_1's auc: 0.874157
[715]	valid_0's auc: 0.88398	valid_1's auc: 0.874156
[716]	valid_0's auc: 0.883986	valid_1's auc: 0.874154
[717]	valid_0's auc: 0.883994	valid_1's auc: 0.874156
[718]	valid_0's auc: 0.884001	valid_1's auc: 0.874149
[719]	valid_0's auc: 0.884009	valid_1's auc: 0.87415
[720]	valid_0's auc: 0.884016	valid_1's auc: 0.874151
[721]	valid_0's auc: 0.884023	valid_

In [50]:
print(np.mean(valid_dataset[valid_dataset.is_prefix_in_train == 1]['predicted_score']))
print(np.mean(valid_dataset[valid_dataset.is_prefix_in_train == 0]['predicted_score']))


0.37169754230398205
0.4559382820494978


In [51]:
#直接搜索最佳阈值

yuzhi_dict = {}
# 定义搜索方法获取最佳F1对应的阈值
for yuzhi in range(350, 700, 2):
    real_yuzhi = yuzhi / 1000
    valid_dataset['predicted_label'] = valid_dataset['predicted_score'].map(lambda x : 1 if x > real_yuzhi else 0)
    f1 = f1_score(valid_dataset['label'], valid_dataset['predicted_label'])
    yuzhi_dict[str(real_yuzhi)] = f1
print(yuzhi_dict)



{'0.35': 0.7382434632698016, '0.352': 0.7381500671656113, '0.354': 0.7385642898327888, '0.356': 0.7391618497109826, '0.358': 0.7391702846116739, '0.36': 0.7393877896161002, '0.362': 0.7392672184308601, '0.364': 0.738782439970895, '0.366': 0.7388241010689991, '0.368': 0.7389313720715016, '0.37': 0.739040277134005, '0.372': 0.7388488329463522, '0.374': 0.7388419635206266, '0.376': 0.738911908206046, '0.378': 0.7391144183305092, '0.38': 0.7388522492371297, '0.382': 0.7390017754981257, '0.384': 0.7388305105699097, '0.386': 0.7393339111056085, '0.388': 0.7395717015310553, '0.39': 0.7395885100884604, '0.392': 0.7396476209436592, '0.394': 0.7394014962593516, '0.396': 0.7395939973532424, '0.398': 0.7395635198718591, '0.4': 0.7396470470269728, '0.402': 0.7395228398320638, '0.404': 0.7399702823179792, '0.406': 0.7401765447667087, '0.408': 0.7401869158878503, '0.41': 0.7403362406775376, '0.412': 0.7404895087195323, '0.414': 0.7402844322762187, '0.416': 0.7402725957511612, '0.418': 0.7403604198968

In [58]:
valid_prefix0_df = valid_dataset[valid_dataset.is_prefix_in_train == 1].copy()

#定义调整函数
def resultAdjustment(result_df, t):
    result_df_temp = result_df.copy()
    result_df_temp['x'] = result_df_temp.predicted_score.map(lambda x: -(math.log(((1 - x) / x), math.e)))
    result_df_temp['adjust_result'] = result_df_temp.x.map(lambda x: 1 / (1 + math.exp(-(x + t)))) 
    print(result_df_temp['adjust_result'].mean())
    return result_df_temp['adjust_result']

print('original mean : ', valid_prefix0_df['predicted_score'].mean())
valid_df_after = resultAdjustment(valid_prefix0_df, 0.61705)



original mean :  0.37169754230398205
0.45584923601204286


In [59]:
valid_dataset['predicted_score'][valid_dataset.is_prefix_in_train == 1] = valid_df_after
print(np.mean(valid_dataset['predicted_score'][valid_dataset.is_prefix_in_train == 0]))
print(np.mean(valid_dataset['predicted_score'][valid_dataset.is_prefix_in_train == 1]))


0.4559382820494978
0.45584923601204286


/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [60]:
yuzhi_dict = {}
# 定义搜索方法获取最佳F1对应的阈值
for yuzhi in range(350, 700, 2):
    real_yuzhi = yuzhi / 1000
    valid_dataset['predicted_label'] = valid_dataset['predicted_score'].map(lambda x : 1 if x > real_yuzhi else 0)
    f1 = f1_score(valid_dataset['label'], valid_dataset['predicted_label'])
    yuzhi_dict[str(real_yuzhi)] = f1
print(yuzhi_dict)



{'0.35': 0.7226496569562263, '0.352': 0.7232386313659952, '0.354': 0.7237210121787389, '0.356': 0.7247753019897091, '0.358': 0.7254142790769705, '0.36': 0.7257217556998206, '0.362': 0.7260663086414273, '0.364': 0.7262343895826853, '0.366': 0.7266212667467841, '0.368': 0.7268512326690741, '0.37': 0.7268910378095896, '0.372': 0.7271344191873993, '0.374': 0.72777416534003, '0.376': 0.7282484226598109, '0.378': 0.728577534813615, '0.38': 0.7288792714813813, '0.382': 0.7292104134209397, '0.384': 0.7296704290981474, '0.386': 0.7305085130018817, '0.388': 0.7311075251710266, '0.39': 0.7316195723946355, '0.392': 0.7320532700903037, '0.394': 0.7321844859898524, '0.396': 0.7325993911789612, '0.398': 0.7328167270392957, '0.4': 0.7333119089156183, '0.402': 0.7335482385447848, '0.404': 0.7343530117424506, '0.406': 0.7349570200573066, '0.408': 0.7353880327641261, '0.41': 0.7357585031647771, '0.412': 0.7363357501394311, '0.414': 0.7365530082625392, '0.416': 0.7366555511508058, '0.418': 0.7369797748610

In [61]:
valid_dataset['predicted_label'] = valid_dataset['predicted_score'].map(lambda x : 1 if x > 0.492 else 0)
print(np.mean(valid_dataset['label'][valid_dataset.is_prefix_in_train == 0]))
print(np.mean(valid_dataset['label'][valid_dataset.is_prefix_in_train == 1]))
print(np.mean(valid_dataset['predicted_label'][valid_dataset.is_prefix_in_train == 0]))
print(np.mean(valid_dataset['predicted_label'][valid_dataset.is_prefix_in_train == 1]))


0.3741522590505301
0.37105056790872987
0.4383417709427835
0.4227568237585692
